In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

In [2]:
# 张量。tf中使用numpy来表示张量值！
# tensorflow core 低阶API
# 主要分为两个互相独立的部分组成：
# - 构建计算图（tf.Graph）
# - 运行计算图（tf.Session）

In [3]:
a = tf.constant(3.0,dtype = tf.float32)
b = tf.constant(4.0)# 不用显示指明张量类型
total = a+b

print(a)
print(b)
print(total)

Tensor("Const:0", shape=(), dtype=float32)
Tensor("Const_1:0", shape=(), dtype=float32)
Tensor("add:0", shape=(), dtype=float32)


In [4]:
# 这里可以看到输出值并不是3.0,4.0,7.0。
# 上述语句构建计算图，tf.tensor仅代表将要运行的操作的结果
# 图中指令都有唯一名词，张量是根据生成他们的指令命名，后面跟着输出索引。

In [5]:
# tensorboard
# 将计算图保存为tensorboard的摘要文件
writer = tf.summary.FileWriter('.')
writer.add_graph(tf.get_default_graph())

In [ ]:
# 这将在当前目录中生成一个 event 文件，其名称格式如下：
# events.out.tfevents.{timestamp}.{hostname}
! tensorboard --logdir .
#http://localhost:6006/#graphs&run=.

In [9]:
# 会话
# 评估张量，需要实例化tf.session对象。会话会封装tf运行时的状态，并运行tf的操作。
# tf.graph比作一个py文件，tf.session比作一个python可执行对象。
sess = tf.Session()
print(sess.run(total))

7.0


In [10]:
# 传递多个张量，并以透明方式处理元组或字典的任何组合。
print(sess.run({'ab':(a,b),'total':total}))

{'ab': (3.0, 4.0), 'total': 7.0}


In [13]:
# 在调用 tf.Session.run 期间，任何 tf.Tensor 都只有单个值
# 每次调用vec，返回不同值，但同一run，vec值相同。
vec = tf.random_uniform(shape=(3,))
out1 = vec + 1
out2 = vec + 2
print(sess.run(vec))
print(sess.run(vec))
print(sess.run((out1, out2)))

[0.70143473 0.580384   0.7526375 ]
[0.47295773 0.4392314  0.2897011 ]
(array([1.9578346, 1.5962874, 1.9934797], dtype=float32), array([2.9578347, 2.5962873, 2.9934797], dtype=float32))


In [14]:
# 供给
# 常量自定义，变量需要初始化，但有的时候我们需要设置参数方便接收外部输入。
# 占位符。即承诺稍后提供值，先占个位子。类似函数参数
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
z = x + y

In [15]:
# run方法中的feed_dict函数用来以字典的形式填入参数
print(sess.run(z,feed_dict={x:3,y:4.5}))
print(sess.run(z,feed_dict={x:[1,3],y:[2,5]}))

7.5
[3. 8.]


In [16]:
# 创建数据集迭代器
my_Data = [
    [0,1],
    [2,3],
    [4,5],
    [6,7],
]
slices = tf.data.Dataset.from_tensor_slices(my_Data)
next_item = slices.make_one_shot_iterator().get_next()

In [17]:
while True:
    try:
        print(sess.run(next_item))
    except tf.errors.OutOfRangeError:
        break

[0 1]
[2 3]
[4 5]
[6 7]


In [28]:
# 如何dataset依赖于有状态操作，需要在使用迭代器前初始化。
# 到达数据流末端时，Dataset 会抛出 OutOfRangeError
r = tf.random_normal([10,3])

dataset = tf.data.Dataset.from_tensor_slices(r)#sess.run(r))
iterator = dataset.make_initializable_iterator()
next_row = iterator.get_next()

sess.run(iterator.initializer)

while True:
    try:
        print(sess.run(next_row))
    except tf.errors.OutOfRangeError:
        #print("error")
        break

[-1.2907134  -0.627311    0.43869826]
[-2.2644894 -1.1150272 -2.147645 ]
[-1.861257   -0.5105497  -0.37195173]
[-0.91531116  1.2565628   0.9942749 ]
[ 0.01453245 -1.3357207   0.87064946]
[-0.6016345   0.82184863 -1.3833783 ]
[ 0.06143059 -0.08581635  1.2042434 ]
[-2.0214412   0.7412764   0.42179057]
[-1.0324953   0.23998797  0.04892275]
[-0.06483631  0.83192784  0.6084466 ]


In [29]:
# 层layer
# 可训练的模型必须修改图中的值，以便在输入相同值的情况下获得新的输出值。
# 将可训练参数添加到图中的首选方法是层。层将变量和操作打包

#创建层
x = tf.placeholder(tf.float32,shape=[None, 3])
linear_model = tf.layers.Dense(units=1)
y = linear_model(x)
# 给定占位符的形状，层会检查输入数据，确定内部变量大小。

In [30]:
# 初始化层，层包含的变量必须先初始化
# 初始化应在构建图表的最后一步添加初始化程序
init = tf.global_variables_initializer()
sess.run(init)

In [31]:
# 执行层
print(sess.run(y,{x:[[1,2,3],[4,5,6]]}))

[[-2.0377107]
 [-6.8450813]]


In [38]:
sess.close()

In [36]:
# 训练
# 定义数据
x = tf.constant([[1],[2],[3],[4]],dtype=tf.float32)
y_true = tf.constant([[0],[-1],[-2],[-3]],dtype=tf.float32)

In [37]:
# 定义模型
linear_model = tf.layers.Dense(units=1)
y = linear_model(x)

In [39]:
# 评估预测值
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

print(sess.run(y))

[[1.3660356]
 [2.7320712]
 [4.098107 ]
 [5.4641423]]


In [40]:
# 优化模型，就要定义损失。tf.losses
loss = tf.losses.mean_squared_error(labels =y_true,predictions=y)
print(sess.run(loss))

31.155754


In [41]:
# 训练，定义优化器使损失最小化。tf.train.Optimizer
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

In [42]:
for i in range(100):
    _, loss_value = sess.run((train, loss))
    print(loss_value)

31.155754
21.749487
15.221891
10.6917515
7.5476084
5.365184
3.85008
2.7980201
2.0672615
1.5594518
1.2063458
0.96059
0.78932655
0.6697564
0.5860595
0.52725863
0.4857367
0.45620883
0.4350075
0.41958818
0.4081851
0.3995729
0.39290157
0.38758093
0.38320187
0.37948006
0.3762185
0.37328026
0.37057057
0.36802334
0.36559284
0.36324736
0.36096475
0.35872972
0.3565316
0.3543629
0.35221857
0.35009488
0.34798938
0.34590018
0.3438262
0.34176636
0.3397201
0.33768696
0.33566666
0.3336588
0.33166325
0.32967982
0.32770836
0.3257488
0.32380104
0.321865
0.31994054
0.31802762
0.31612617
0.3142361
0.3123573
0.31048974
0.30863333
0.30678812
0.30495384
0.30313057
0.30131823
0.29951665
0.2977259
0.29594588
0.2941764
0.2924176
0.29066932
0.28893137
0.2872039
0.28548673
0.28377986
0.2820832
0.28039667
0.27872023
0.2770538
0.27539736
0.2737508
0.27211404
0.27048713
0.2688699
0.26726243
0.2656645
0.26407614
0.26249728
0.26092786
0.25936776
0.2578171
0.25627562
0.25474337
0.25322032
0.25170633
0.2502014
0.2487055


In [43]:
# 由于train是一直指令，而loss是一个张量，我们同时运行，以实时查看损失
print(sess.run(y))

[[-0.79089534]
 [-1.383243  ]
 [-1.9755905 ]
 [-2.567938  ]]
